## Overview
- The functions used here are probably going to be used the most, but there are some other functions that I have also written in the py file. There is some documentation on each function in the py file as well. 
- Although finicky, the Youtube API is extremely powerful data collection tool and can give you access to other data I might not have included in my functions. Feel free to look at this documentation, https://developers.google.com/youtube/v3/docs, if you want to learn more! It is pretty simple to alter the code I have already written to pull other data, especially if you take a look at some of the smaller functions I used to build the functions utilized in this template
- I initially followed this tutorial to learn more about the API if you wanted to learn how I wrote these functions! https://www.youtube.com/watch?v=2mSwcRb3KjQ

In [1]:
import numpy as np
import pandas as pd
from sam_fun import *

## 1) Get your own YT API key
- Use the directions in this link to get your own API key you'll need to access the YT API
- https://blog.hubspot.com/website/how-to-get-youtube-api-key

In [4]:
#add your api key
api_key = ""
from googleapiclient.discovery import build
youtube = build("youtube","v3",developerKey=api_key)

## 2) Getting Channel IDs
- Each channel on Youtube has a unique channel ID
- the function "trending_creators_by_country" takes in a country code, and returns a list of channel IDs from the specified country's trending page
- If you want to get a specifc Youtube channel's ID, 
     1. Go to their Youtube Page
     2. Right click and hit view page sources
     3. Command_F browse_id, ID is stored in value key next to it (ex. Bon Appetit's is UCbpMy0Fg74eXXkvxJrtEn3w)
- To find other country codes, look at this link https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2 
    - IN is India

In [20]:
#top creators from india's trending video page
#leave youtube as your parameter, this is just to access the API
in_creator_ids= trending_creators_by_country(youtube,"IN")
in_creator_ids[0:5]

['UCu1cFPQalIQlHHDSxn_PI0w',
 'UCjF5ecCYXFDvR0UY9GcfnoA',
 'UCOcD_2N04_XS6XgImIWcR1Q',
 'UCXS0LXj5BO9CS8uVBFPBvrg',
 'UC9cpaBjhse6B76HP_gn4Rew']

## 3) Channel Stats
- "channel_stats" will return a data frame containing overall channel stats, including total view count across all their videos, sub count, and video count
- the function takes in a list of channel IDs

In [21]:
#dataframe of stats from creators, made with the list of ids
in_stats_df = channels_stats(youtube,in_creator_ids)

In [22]:
in_stats_df

,title,country,viewCount,subscriberCount,videoCount,channel_id
0,AK Entertainments,IN,348988837,695000,668,UCu1cFPQalIQlHHDSxn_PI0w
1,Cinema Vikatan,IN,1770975914,3990000,8623,UCjF5ecCYXFDvR0UY9GcfnoA
2,MR. INDIAN HACKER Vlogs,IN,137601665,1900000,111,UCOcD_2N04_XS6XgImIWcR1Q
3,Junglee Music Tamil,IN,303590018,655000,531,UCXS0LXj5BO9CS8uVBFPBvrg
4,Dora Sai Teja,IN,179988133,786000,175,UC9cpaBjhse6B76HP_gn4Rew
...,...,...,...,...,...,...
113,JSS GAMER,IN,143690169,934000,589,UC3sZ9ZahiCGfLAUdVIVv8CA
114,Shruti Arjun Anand,IN,2394819270,10100000,735,UCIsotCCdTjStPCLyD9iBCCQ
115,Colors TV,IN,58087002541,61500000,107305,UC55IWqFLDH1Xp7iu1_xknRA
116,Dreamz Unlimited,IN,210451613,870000,281,UC7A2r9_k7hBfAmUz7F5flag


## 3) Filtering Channels
- the function "top_channels" takes in a dataframe made in the previous function, and finds the top channels based on a specified column
- In this case, the top 5% of channels based on their number of subs

In [ ]:
# gets top 5% of channels based off subscribers
in_95 = top_channels(in_stats_df,"subscriberCount",95)

## 4) Video IDs
- Similar to how each channel has a unique ID, each video also has a unique ID. From the video ID, you can get video title, publish date, description, tags, view counts for that specifc video, dislike count (if not made private), comment count, whether it was made for kids, and much more (that I have not included in this function). 
- get_videoID_list takes in a channel ID and returns their entire video collection in the form of a list of video IDs

In [26]:
# gets id of every video on khordha toka channel
video_ids = get_videoID_list(youtube,"UCmuSPlchjLraieuEs33mcVA" ) # or in_stats_df[2] 


In [28]:
video_ids[0:5]

['rkpINIPddb4', 'JiMdYa0wuaQ', '-4R0HtJ9V40', 'ULW_HnyPr5E', '7gfJPhUR4Vw']

## 5) Video Stats
- the function "get_video_details" takes in a list of video IDs and returns a data frame on stats from every video. 
- In this example, I used the list from above to get stats on every video a channel has posted
- From here you can get things like average like count or most popular tags.

In [29]:
# dataframe containing stats of all videos of a channel
get_video_details(youtube,video_ids)

,title,published,description,tags,viewCount,dislikeCount,commentCount,postingDate,made_for_kids
0,Ama Gaan huli khela || Khordha toka || Funny a...,2023-03-06T11:27:24Z,Download probo 👇\nhttps://probo-in.onelink.me/...,"[ama gaan huli khela, khordha toka comedy, ama...",4388974,private,3084,2023-03-06T11:27:24Z,False
1,Chala jiba picnic || FULL VDO || BHAI THE MOV...,2023-01-06T14:23:38Z,Brand promotion by :- @the_lucky_goyal \nKhord...,"[chala jiba picnic, khordhatoka, funny anuguli...",3410105,private,3345,2023-01-06T14:23:38Z,False
2,chala jiba PICNIC || TEASER || Khordha toka |...,2022-12-26T10:36:34Z,,"[khordha toka, Anguilla, mr dhenkanaliaa, khor...",204790,private,294,2022-12-26T10:36:34Z,False
3,Ama ghara katha PART3 || Khordha toka || Funny...,2022-11-05T12:03:54Z,Download probo 👇\nhttps://probo-in.onelink.me/...,"[ama ghara katha, khordha toka comedy, funny a...",4890355,private,2452,2022-11-05T12:03:54Z,False
4,Puja re lagila 144 || Khordha toka || Funny an...,2022-09-21T10:21:25Z,Click for FREE registration on Olymp Trade: ht...,"[ganesh puja impossible, khordha toka comedy, ...",1800277,private,3091,2022-09-21T10:21:25Z,False
...,...,...,...,...,...,...,...,...,...
63,Ame Samaste ek//. Odia motivational video //CR...,2018-02-12T10:06:30Z,,"[ame samaste ek, Ame samaste ek, AME SAMASTE E...",36919,private,192,2018-02-12T10:06:30Z,False
64,YAARA TERI YAARI..😢😢// CRaZyyy BOYZZ,2018-01-17T08:00:44Z,,"[yaara teri yaari, YAARA TERI YAARI, Yaara Ter...",22144,private,164,2018-01-17T08:00:44Z,False
65,ODIA Advertisement VS REALITY || By CRazzY B...,2018-01-02T10:59:47Z,,"[Odia advertisement vs reality, odia ADVERTISE...",111289,private,168,2018-01-02T10:59:47Z,False
66,BAPA || a. Heart touching short film||,2017-12-19T08:38:06Z,,[],26463,private,208,2017-12-19T08:38:06Z,False
